In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive




---



**AFTER THIS POINT YOU CAN START WORKING**

---








In [ ]:
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category20c
from bokeh.layouts import column, row
from bokeh.transform import cumsum, linear_cmap
import warnings
warnings.filterwarnings('ignore')

# Read the csv file and drop nulls, invlid columns and duplicates
df = pd.read_csv("/content/gdrive/MyDrive/DataAnalysisTasks/Datasets/startup_data.csv")
df.dropna(inplace=True)
df.drop(columns = ['Unnamed: 6',"Unnamed: 0"],inplace=True)

# Sort dataframe to bring the top funded companies to top
df1 = df.sort_values(by='funding_total_usd', ascending = False)

# Trimming zip code for only first 5 digits
df1['zip_code'] = df1['zip_code'].str[:5]
df1 = df1.drop_duplicates(subset='zip_code', keep='first')

# Top 20 funding zip codes
df1 = df1[['zip_code','funding_total_usd']][:20]
scatter_data = df1.copy()
zip_codes = scatter_data['zip_code'].unique()

# Incorporate colors
palette = Category20c[len(zip_codes)]
colormap = {zip_codes[i]: palette[i] for i in range(len(zip_codes))}
scatter_data['colors'] = [colormap[x] for x in scatter_data['zip_code']]

# Create a figure and add a number of tools
p = figure(height=800, width=800, title = "zip_code vs funding_total_usd",
           tools="pan,wheel_zoom,box_zoom,reset,hover,save",
           tooltips="Zip Code = @zip_code - funding total in USD = @funding_total_usd, Port = @funding_total_usd",
           toolbar_location="below")

# format our data in a ColumnDataSource format
data = ColumnDataSource(scatter_data)

# Add scatter object
p.scatter("zip_code", "funding_total_usd", source=data, legend_field="zip_code", fill_alpha=0.8, size=7,
          fill_color='colors', line_color='colors')

# Format axes
p.xaxis.axis_label = 'zip_code'
p.yaxis.axis_label = 'Funding in USD'

show(p)
